In [51]:
#Установка catboost
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [52]:
#import необходимых модулей

import pandas as pd
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

import numpy as np
import math
from datetime import datetime, timedelta

In [53]:
#Считывание данных в DataFrame 
test = pd.read_csv('test_dataset_test.csv', sep=';', index_col=None,dtype={'PATIENT_SEX':str, 'MKB_CODE':str, 'ADRES':str, 'VISIT_MONTH_YEAR':str, 'AGE_CATEGORY':str})

train_1_50 = pd.read_csv('train_1_50.csv', sep=';', index_col=None)
train_over_50 = pd.read_csv('train_over_50.csv', sep=';', index_col=None)

test_1_50 = pd.read_csv('test_1_50.csv', sep=';', index_col=None)
test_over_50 = pd.read_csv('test_over_50.csv', sep=';', index_col=None)

test_success = pd.read_csv('test_success_optimal.csv', sep=';', index_col=None)

In [54]:
test_success

,id,MKB_CODE,ADRES,VISIT_MONTH_YEAR,AGE_CATEGORY,PATIENT_SEX,PATIENT_ID_COUNT
0,0,A00,Калининград,22.04.2022,children,0,4
1,20,A04.9,Калининград,22.04.2022,middleage,0,1
2,34,A07.1,Калининград,22.04.2022,old,0,3
3,65,A08.1,Черняховск,22.04.2022,children,0,1
4,84,A09,Калининград,22.04.2022,young,0,1
...,...,...,...,...,...,...,...
29900,10436,K07.3,Черняховск,?,children,0,32
29901,21326,Z01.0,Черняховск,?,children,0,37
29902,29327,J02.8,Черняховск,?,children,1,24
29903,29517,J06.8,Черняховск,?,children,1,171


In [55]:
test_1_50

,MAX_PATIENT_ID_COUNT,id,MKB_CODE,ADRES,AGE_CATEGORY,MKB_group,LAST_PATIENT_ID_COUNT,PATIENT_ID_COUNT,Mean,Mediana,Stotkl,last,MIN_PATIENT_ID_COUNT,Median_PATIENT_ID_COUNT,COL_PATIENT_ZAP,MEAN_PATIENT,PATIENT_SEX
0,26,19,A04.9,Калининград,children,A,4,3,3.083333,3.0,1.443376,4,1,6.0,51,7.058824,0
1,24,23111,A04.9,Калининград,children,A,5,6,5.200000,5.0,3.047768,5,1,7.0,49,8.469388,1
2,15,21,A04.9,Калининград,young,A,1,5,2.666667,1.0,2.061553,1,1,4.0,48,5.000000,0
3,13,23114,A04.9,Калининград,young,A,3,2,2.363636,2.0,1.501514,3,1,4.0,47,4.127660,1
4,6,23116,A04.9,Черняховск,children,A,1,1,3.000000,3.5,1.851640,1,1,1.0,15,2.200000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7904,17,23052,Z85.4,Калининград,middleage,Z,1,2,2.333333,2.0,1.658312,1,1,4.0,33,5.272727,0
7905,6,23074,Z93.3,Калининград,elderly,Z,3,5,1.888889,1.0,1.364225,3,1,1.0,21,1.904762,0
7906,8,39348,Z93.3,Калининград,elderly,Z,8,1,2.777778,2.0,2.279132,8,1,1.0,24,1.875000,1
7907,3,39349,Z93.3,Калининград,middleage,Z,3,1,1.285714,1.0,0.755929,3,1,1.0,14,1.142857,1


In [56]:
train_1_50

,MAX_PATIENT_ID_COUNT,MKB_CODE,ADRES,AGE_CATEGORY,id,MKB_group,LAST_PATIENT_ID_COUNT,PATIENT_ID_COUNT,Mean,Mediana,Stotkl,last,MIN_PATIENT_ID_COUNT,Median_PATIENT_ID_COUNT,COL_PATIENT_ZAP,MEAN_PATIENT,PATIENT_SEX
0,26,A04.9,Калининград,children,19,A,4,3,3.333333,3.0,1.723281,5,1,6.0,51,7.058824,0
1,24,A04.9,Калининград,children,23111,A,5,6,6.583333,5.0,4.273775,6,1,7.0,49,8.469388,1
2,15,A04.9,Калининград,young,21,A,1,5,2.916667,3.0,1.621354,1,1,4.0,48,5.000000,0
3,13,A04.9,Калининград,young,23114,A,3,2,2.900000,2.0,2.378141,1,1,4.0,47,4.127660,1
4,6,A04.9,Черняховск,children,23116,A,1,1,1.666667,2.0,0.577350,1,1,1.0,15,2.200000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7904,17,Z85.4,Калининград,middleage,23052,Z,1,2,3.111111,4.0,1.452966,2,1,4.0,33,5.272727,0
7905,6,Z93.3,Калининград,elderly,23074,Z,3,5,1.666667,2.0,0.577350,1,1,1.0,21,1.904762,0
7906,8,Z93.3,Калининград,elderly,39348,Z,8,1,1.142857,1.0,0.377964,1,1,1.0,24,1.875000,1
7907,3,Z93.3,Калининград,middleage,39349,Z,3,1,1.000000,1.0,0.000000,1,1,1.0,14,1.142857,1


In [57]:
train_over_50

,MAX_PATIENT_ID_COUNT,MKB_CODE,ADRES,AGE_CATEGORY,id,MKB_group,LAST_PATIENT_ID_COUNT,PATIENT_ID_COUNT,Mean,Mediana,Stotkl,last,MIN_PATIENT_ID_COUNT,Median_PATIENT_ID_COUNT,COL_PATIENT_ZAP,MEAN_PATIENT,PATIENT_SEX
0,59,A09,Калининград,children,81,A,8,6,4.100000,5.5,2.514403,6,1,8.0,49,14.897959,0
1,67,A09,Калининград,children,23164,A,9,12,6.000000,5.0,4.381780,10,1,9.5,50,17.160000,1
2,107,A63.8,Калининград,young,167,A,13,31,35.750000,37.0,11.466354,31,13,31.0,51,34.686275,0
3,68,B01,Калининград,children,23295,B,2,14,6.545455,7.0,5.627853,18,1,9.0,49,13.040816,1
4,142,B01.9,Гурьевск,children,217,B,88,142,21.833333,9.5,30.809778,110,1,9.0,50,21.080000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1705,60,Z76.0,Калининград,young,23013,Z,60,18,23.250000,18.5,11.434597,15,1,18.0,44,19.409091,0
1706,53,Z76.0,Калининград,young,39302,Z,47,16,17.416667,18.0,5.791190,18,1,16.0,43,16.558140,1
1707,109,Z76.0,Советск,elderly,23026,Z,16,78,9.166667,6.5,5.812734,21,1,10.0,46,22.369565,0
1708,60,Z76.0,Советск,elderly,39316,Z,4,35,8.333333,7.5,4.978743,17,1,8.0,43,14.860465,1


In [58]:
test_over_50

,MAX_PATIENT_ID_COUNT,id,MKB_CODE,ADRES,AGE_CATEGORY,MKB_group,LAST_PATIENT_ID_COUNT,PATIENT_ID_COUNT,Mean,Mediana,Stotkl,last,MIN_PATIENT_ID_COUNT,Median_PATIENT_ID_COUNT,COL_PATIENT_ZAP,MEAN_PATIENT,PATIENT_SEX
0,59,81,A09,Калининград,children,A,8,6,6.583333,6.5,2.020726,8,1,8.0,49,14.897959,0
1,67,23164,A09,Калининград,children,A,9,12,7.750000,7.0,4.245318,9,1,9.5,50,17.160000,1
2,68,23295,B01,Калининград,children,B,2,14,5.636364,3.0,5.005452,2,1,9.0,49,13.040816,1
3,142,217,B01.9,Гурьевск,children,B,88,142,45.000000,23.0,49.149124,88,1,9.0,50,21.080000,0
4,173,23272,B01.9,Гурьевск,children,B,87,173,47.250000,24.0,51.351064,87,1,12.5,46,23.804348,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1554,60,23013,Z76.0,Калининград,young,Z,60,18,27.500000,23.0,14.563341,60,1,18.0,44,19.409091,0
1555,53,39302,Z76.0,Калининград,young,Z,47,16,24.833333,18.5,14.313588,47,1,16.0,43,16.558140,1
1556,109,23026,Z76.0,Советск,elderly,Z,16,78,65.250000,75.5,33.984288,16,1,10.0,46,22.369565,0
1557,60,39316,Z76.0,Советск,elderly,Z,4,35,35.333333,39.0,19.703868,4,1,8.0,43,14.860465,1


In [59]:
# выбираем поля для нейросетей
X_train_1_50=train_1_50[['PATIENT_SEX', 'ADRES', 'AGE_CATEGORY','MKB_group','Mean',	'Mediana',	'Stotkl',	'last']]
y_train_1_50=train_1_50[['PATIENT_ID_COUNT']]

testnet_1_50=test_1_50[['PATIENT_SEX', 'ADRES', 'AGE_CATEGORY','MKB_group',	'Mean','Mediana',	'Stotkl',	'last']]

X_train_over_50=train_over_50[['PATIENT_SEX', 'ADRES', 'AGE_CATEGORY','MKB_group','Mean',	'Mediana',	'Stotkl',	'last']]
y_train_over_50=train_over_50[['PATIENT_ID_COUNT']]

testnet_over_50=test_over_50[['PATIENT_SEX', 'ADRES', 'AGE_CATEGORY','MKB_group','Mean',	'Mediana',	'Stotkl',	'last']]






In [60]:
testnet_over_50

,PATIENT_SEX,ADRES,AGE_CATEGORY,MKB_group,Mean,Mediana,Stotkl,last
0,0,Калининград,children,A,6.583333,6.5,2.020726,8
1,1,Калининград,children,A,7.750000,7.0,4.245318,9
2,1,Калининград,children,B,5.636364,3.0,5.005452,2
3,0,Гурьевск,children,B,45.000000,23.0,49.149124,88
4,1,Гурьевск,children,B,47.250000,24.0,51.351064,87
...,...,...,...,...,...,...,...,...
1554,0,Калининград,young,Z,27.500000,23.0,14.563341,60
1555,1,Калининград,young,Z,24.833333,18.5,14.313588,47
1556,0,Советск,elderly,Z,65.250000,75.5,33.984288,16
1557,1,Советск,elderly,Z,35.333333,39.0,19.703868,4


In [61]:
#Обучаем для заболеваний 1_50

pool_train_solution = Pool(X_train_1_50, y_train_1_50, cat_features = ['PATIENT_SEX',  'ADRES', 'AGE_CATEGORY','MKB_group',])
pool_test_solution = Pool(testnet_1_50,cat_features = ['PATIENT_SEX',  'ADRES', 'AGE_CATEGORY','MKB_group',])
                       
model_test_1_50 = CatBoostRegressor(iterations=1200,  task_type='GPU',depth=12, verbose =True, random_seed=15)
model_test_1_50.fit(pool_train_solution)

Learning rate set to 0.0475
0:	learn: 7.0816126	total: 137ms	remaining: 2m 44s
1:	learn: 6.8597762	total: 315ms	remaining: 3m 8s
2:	learn: 6.6474452	total: 696ms	remaining: 4m 37s
3:	learn: 6.4456122	total: 1.01s	remaining: 5m
4:	learn: 6.2557658	total: 1.3s	remaining: 5m 9s
5:	learn: 6.0751727	total: 1.42s	remaining: 4m 43s
6:	learn: 5.9043861	total: 1.71s	remaining: 4m 51s
7:	learn: 5.7468232	total: 2.15s	remaining: 5m 20s
8:	learn: 5.5974334	total: 2.45s	remaining: 5m 24s
9:	learn: 5.4593857	total: 2.75s	remaining: 5m 27s
10:	learn: 5.3332359	total: 3.05s	remaining: 5m 29s
11:	learn: 5.2180104	total: 3.21s	remaining: 5m 17s
12:	learn: 5.1144091	total: 3.22s	remaining: 4m 54s
13:	learn: 5.0117653	total: 3.27s	remaining: 4m 36s
14:	learn: 4.9167075	total: 3.34s	remaining: 4m 23s
15:	learn: 4.8202989	total: 3.59s	remaining: 4m 25s
16:	learn: 4.7350952	total: 3.73s	remaining: 4m 19s
17:	learn: 4.6582232	total: 3.81s	remaining: 4m 10s
18:	learn: 4.5814169	total: 4.08s	remaining: 4m 13s
1

In [62]:
# предварительный тест схождения на всем обучающем множестве
y_pred = model_test_1_50.predict(X_train_1_50)
y_pred

array([4.22185894, 5.96869101, 2.89293404, ..., 1.93636604, 1.8799551 ,
       3.46195583])

In [63]:
print("Значение метрики R2 на трейне 1_50: ", r2_score(y_train_1_50, y_pred))

Значение метрики R2 на трейне 1_50:  0.8119270741907667


In [64]:
#Получение ответов
y_pred_solution_1_50 = model_test_1_50.predict(pool_test_solution)
y_pred_solution_1_50

array([3.58582182, 5.26628988, 3.6519065 , ..., 5.93644939, 2.32596608,
       6.0854007 ])

In [65]:
#Округляем прогнозы
y_pred_solution_1_50_round=np.round(y_pred_solution_1_50, 0)

y_pred_solution_1_50_round=y_pred_solution_1_50_round.astype(int)

In [66]:
# Заводим результаты в тест
test_1_50['PATIENT_ID_COUNT'] = y_pred_solution_1_50_round

test_1_50

,MAX_PATIENT_ID_COUNT,id,MKB_CODE,ADRES,AGE_CATEGORY,MKB_group,LAST_PATIENT_ID_COUNT,PATIENT_ID_COUNT,Mean,Mediana,Stotkl,last,MIN_PATIENT_ID_COUNT,Median_PATIENT_ID_COUNT,COL_PATIENT_ZAP,MEAN_PATIENT,PATIENT_SEX
0,26,19,A04.9,Калининград,children,A,4,4,3.083333,3.0,1.443376,4,1,6.0,51,7.058824,0
1,24,23111,A04.9,Калининград,children,A,5,5,5.200000,5.0,3.047768,5,1,7.0,49,8.469388,1
2,15,21,A04.9,Калининград,young,A,1,4,2.666667,1.0,2.061553,1,1,4.0,48,5.000000,0
3,13,23114,A04.9,Калининград,young,A,3,3,2.363636,2.0,1.501514,3,1,4.0,47,4.127660,1
4,6,23116,A04.9,Черняховск,children,A,1,3,3.000000,3.5,1.851640,1,1,1.0,15,2.200000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7904,17,23052,Z85.4,Калининград,middleage,Z,1,3,2.333333,2.0,1.658312,1,1,4.0,33,5.272727,0
7905,6,23074,Z93.3,Калининград,elderly,Z,3,3,1.888889,1.0,1.364225,3,1,1.0,21,1.904762,0
7906,8,39348,Z93.3,Калининград,elderly,Z,8,6,2.777778,2.0,2.279132,8,1,1.0,24,1.875000,1
7907,3,39349,Z93.3,Калининград,middleage,Z,3,2,1.285714,1.0,0.755929,3,1,1.0,14,1.142857,1


In [67]:
test_1_50.to_csv('test_1_50_predict.csv', sep=';', index=None)

In [68]:
# Просматриваем результаты
test_1_50['PATIENT_ID_COUNT'].value_counts()

3     1577
2     1210
4      985
5      681
6      531
7      437
8      321
9      266
10     215
11     193
12     171
14     145
13     126
16     114
15     112
18      98
17      92
19      67
20      55
21      52
23      40
25      39
28      38
24      35
22      35
27      34
29      31
26      31
30      25
34      20
35      19
31      17
32      16
33      16
1       15
36      13
37      12
38       9
39       4
41       4
43       3
40       2
44       2
42       1
Name: PATIENT_ID_COUNT, dtype: int64

In [69]:
test_1_50.loc[test_1_50['PATIENT_ID_COUNT']<1]

,MAX_PATIENT_ID_COUNT,id,MKB_CODE,ADRES,AGE_CATEGORY,MKB_group,LAST_PATIENT_ID_COUNT,PATIENT_ID_COUNT,Mean,Mediana,Stotkl,last,MIN_PATIENT_ID_COUNT,Median_PATIENT_ID_COUNT,COL_PATIENT_ZAP,MEAN_PATIENT,PATIENT_SEX


In [70]:
# Клипуем выходящих за предел интервала (снизу, так как заходы вверх не исключены)
test_1_50.loc[(test_1_50['PATIENT_ID_COUNT']<1),'PATIENT_ID_COUNT']=1


In [71]:
test_1_50.to_csv('test_1_50_proba.csv', sep=';', index=None)

In [72]:
test_over_50

,MAX_PATIENT_ID_COUNT,id,MKB_CODE,ADRES,AGE_CATEGORY,MKB_group,LAST_PATIENT_ID_COUNT,PATIENT_ID_COUNT,Mean,Mediana,Stotkl,last,MIN_PATIENT_ID_COUNT,Median_PATIENT_ID_COUNT,COL_PATIENT_ZAP,MEAN_PATIENT,PATIENT_SEX
0,59,81,A09,Калининград,children,A,8,6,6.583333,6.5,2.020726,8,1,8.0,49,14.897959,0
1,67,23164,A09,Калининград,children,A,9,12,7.750000,7.0,4.245318,9,1,9.5,50,17.160000,1
2,68,23295,B01,Калининград,children,B,2,14,5.636364,3.0,5.005452,2,1,9.0,49,13.040816,1
3,142,217,B01.9,Гурьевск,children,B,88,142,45.000000,23.0,49.149124,88,1,9.0,50,21.080000,0
4,173,23272,B01.9,Гурьевск,children,B,87,173,47.250000,24.0,51.351064,87,1,12.5,46,23.804348,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1554,60,23013,Z76.0,Калининград,young,Z,60,18,27.500000,23.0,14.563341,60,1,18.0,44,19.409091,0
1555,53,39302,Z76.0,Калининград,young,Z,47,16,24.833333,18.5,14.313588,47,1,16.0,43,16.558140,1
1556,109,23026,Z76.0,Советск,elderly,Z,16,78,65.250000,75.5,33.984288,16,1,10.0,46,22.369565,0
1557,60,39316,Z76.0,Советск,elderly,Z,4,35,35.333333,39.0,19.703868,4,1,8.0,43,14.860465,1


In [73]:
# Сеть по over 50
pool_train_solution = Pool(X_train_over_50, y_train_over_50, cat_features = ['PATIENT_SEX',  'ADRES', 'AGE_CATEGORY','MKB_group'])
pool_test_solution = Pool(testnet_over_50,cat_features = ['PATIENT_SEX','ADRES', 'AGE_CATEGORY','MKB_group'])
                       
model_test_over_50 = CatBoostRegressor( iterations=1300, task_type='GPU',depth=12, verbose =True, random_seed=34)
model_test_over_50.fit(pool_train_solution)

Learning rate set to 0.037052
0:	learn: 357.4771379	total: 87.4ms	remaining: 1m 53s
1:	learn: 349.5024099	total: 427ms	remaining: 4m 37s
2:	learn: 342.3141473	total: 445ms	remaining: 3m 12s
3:	learn: 334.4658655	total: 479ms	remaining: 2m 35s
4:	learn: 326.7025401	total: 511ms	remaining: 2m 12s
5:	learn: 320.2086454	total: 780ms	remaining: 2m 48s
6:	learn: 312.9335860	total: 1.05s	remaining: 3m 14s
7:	learn: 305.9571502	total: 1.32s	remaining: 3m 33s
8:	learn: 299.1980705	total: 1.4s	remaining: 3m 20s
9:	learn: 292.7400699	total: 1.42s	remaining: 3m 2s
10:	learn: 286.1468645	total: 1.64s	remaining: 3m 11s
11:	learn: 279.8491657	total: 1.97s	remaining: 3m 31s
12:	learn: 273.9805517	total: 1.99s	remaining: 3m 17s
13:	learn: 268.2705451	total: 2.04s	remaining: 3m 7s
14:	learn: 262.5653647	total: 2.32s	remaining: 3m 18s
15:	learn: 256.9519217	total: 2.58s	remaining: 3m 26s
16:	learn: 251.6308630	total: 2.69s	remaining: 3m 23s
17:	learn: 246.5257635	total: 2.71s	remaining: 3m 13s
18:	learn:

In [74]:
# предварительный тест схождения на всем обучающем множестве
y_pred = model_test_over_50.predict(X_train_over_50)
y_pred

array([ 7.15927921,  9.18215441, 25.53729799, ..., 46.42665104,
       38.3885787 , 58.00269352])

In [75]:
print("Значение метрики R2 на трейне over_50: ", r2_score(y_train_over_50, y_pred))

Значение метрики R2 на трейне over_50:  0.9839735404270372


In [76]:
#Получение ответов
y_pred_solution_over_50 = model_test_over_50.predict(pool_test_solution)
y_pred_solution_over_50

array([ 8.14005332,  9.09164996,  2.64885857, ..., 54.57314811,
       29.95611722, 38.47038152])

In [77]:
#Округляем прогнозы
y_pred_solution_over_50_round=np.round(y_pred_solution_over_50, 0)

y_pred_solution_over_50_round

array([ 8.,  9.,  3., ..., 55., 30., 38.])

In [78]:
# Заводим результаты в тест
test_over_50 = pd.read_csv('test_over_50.csv', sep=';', index_col=None)
test_over_50['PATIENT_ID_COUNT'] = y_pred_solution_over_50_round
test_over_50

,MAX_PATIENT_ID_COUNT,id,MKB_CODE,ADRES,AGE_CATEGORY,MKB_group,LAST_PATIENT_ID_COUNT,PATIENT_ID_COUNT,Mean,Mediana,Stotkl,last,MIN_PATIENT_ID_COUNT,Median_PATIENT_ID_COUNT,COL_PATIENT_ZAP,MEAN_PATIENT,PATIENT_SEX
0,59,81,A09,Калининград,children,A,8,8.0,6.583333,6.5,2.020726,8,1,8.0,49,14.897959,0
1,67,23164,A09,Калининград,children,A,9,9.0,7.750000,7.0,4.245318,9,1,9.5,50,17.160000,1
2,68,23295,B01,Калининград,children,B,2,3.0,5.636364,3.0,5.005452,2,1,9.0,49,13.040816,1
3,142,217,B01.9,Гурьевск,children,B,88,98.0,45.000000,23.0,49.149124,88,1,9.0,50,21.080000,0
4,173,23272,B01.9,Гурьевск,children,B,87,103.0,47.250000,24.0,51.351064,87,1,12.5,46,23.804348,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1554,60,23013,Z76.0,Калининград,young,Z,60,51.0,27.500000,23.0,14.563341,60,1,18.0,44,19.409091,0
1555,53,39302,Z76.0,Калининград,young,Z,47,50.0,24.833333,18.5,14.313588,47,1,16.0,43,16.558140,1
1556,109,23026,Z76.0,Советск,elderly,Z,16,55.0,65.250000,75.5,33.984288,16,1,10.0,46,22.369565,0
1557,60,39316,Z76.0,Советск,elderly,Z,4,30.0,35.333333,39.0,19.703868,4,1,8.0,43,14.860465,1


In [79]:
test_over_50.to_csv('test_over_50_predict.csv', sep=';', index=None)

In [80]:
# Просматриваем результаты
test_over_50['PATIENT_ID_COUNT'].value_counts()

50.0     28
56.0     24
43.0     23
48.0     23
19.0     23
         ..
305.0     1
606.0     1
168.0     1
144.0     1
235.0     1
Name: PATIENT_ID_COUNT, Length: 287, dtype: int64

In [81]:
test_over_50.loc[test_over_50['PATIENT_ID_COUNT']<1]

,MAX_PATIENT_ID_COUNT,id,MKB_CODE,ADRES,AGE_CATEGORY,MKB_group,LAST_PATIENT_ID_COUNT,PATIENT_ID_COUNT,Mean,Mediana,Stotkl,last,MIN_PATIENT_ID_COUNT,Median_PATIENT_ID_COUNT,COL_PATIENT_ZAP,MEAN_PATIENT,PATIENT_SEX
297,52,28795,I67.8,Калининград,young,I,1,-0.0,13.833333,7.5,15.798926,1,1,15.0,51,14.627451,1
355,69,29323,J02.8,Калининград,children,J,2,-0.0,2.700000,2.0,1.766981,2,1,5.0,48,15.562500,1
367,85,29344,J02.9,Калининград,elderly,J,2,-3.0,8.833333,7.5,5.524052,2,2,18.0,51,22.529412,1
553,201,29656,J12.9,Калининград,middleage,J,3,-1.0,17.909091,16.0,12.684278,3,1,18.0,27,32.370370,1
555,69,9137,J12.9,Калининград,young,J,2,-2.0,11.583333,8.5,10.130587,2,1,9.0,30,15.333333,0
559,80,9186,J18.9,Гурьевск,elderly,J,1,-3.0,7.333333,8.0,3.393398,1,1,4.0,35,10.400000,0
872,53,15988,M95.8,Калининград,young,M,3,-1.0,2.000000,2.0,1.000000,3,1,6.0,46,9.978261,0
973,55,18234,O23.0,Калининград,young,O,3,0.0,2.100000,2.0,0.875595,3,1,13.0,49,14.714286,0
1041,311,37532,T15.0,Калининград,young,T,2,-0.0,3.833333,3.5,2.405801,2,1,86.0,51,85.705882,1
1288,109,38410,Z02.5,Калининград,young,Z,2,-2.0,2.000000,2.0,1.000000,2,1,29.0,43,30.372093,1


In [82]:
# Клипуем выходящих за предел интервала (снизу, так как заходы вверх не исключены)
test_over_50.loc[(test_over_50['PATIENT_ID_COUNT']<1),'PATIENT_ID_COUNT']=1

In [83]:
# Объединяем все прогнозы в общий предикт
test_all=pd.concat([test_success, test_1_50,test_over_50], ignore_index=True)

In [84]:
# Отбираем поля и сортируем по id, просматриваем
test_all=test_all[['id','PATIENT_SEX','MKB_CODE','ADRES','VISIT_MONTH_YEAR','AGE_CATEGORY','PATIENT_ID_COUNT']]
test_all=test_all.sort_values(by='id')
test_all=test_all.reset_index(drop=True)
test_all['PATIENT_ID_COUNT']=test_all['PATIENT_ID_COUNT'].astype(int)
test_all

,id,PATIENT_SEX,MKB_CODE,ADRES,VISIT_MONTH_YEAR,AGE_CATEGORY,PATIENT_ID_COUNT
0,0,0,A00,Калининград,22.04.2022,children,4
1,1,0,A00,Калининград,22.04.2022,elderly,1
2,2,0,A00,Калининград,22.04.2022,middleage,1
3,3,0,A00,Калининград,22.04.2022,young,3
4,4,0,A01,Калининград,22.04.2022,middleage,1
...,...,...,...,...,...,...,...
39368,39368,1,Z96.6,Балтийск,22.04.2022,elderly,1
39369,39369,1,Z96.6,Гусев,22.04.2022,middleage,1
39370,39370,1,Z96.7,Гусев,22.04.2022,young,1
39371,39371,1,Z98.8,Озерск,22.04.2022,children,1


In [85]:
# Удаляем id и смотрим состав
test_all=test_all[['PATIENT_ID_COUNT']]
test_all=test.join(test_all)
test_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39373 entries, 0 to 39372
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   PATIENT_SEX       39373 non-null  object
 1   MKB_CODE          39373 non-null  object
 2   ADRES             39373 non-null  object
 3   VISIT_MONTH_YEAR  39373 non-null  object
 4   AGE_CATEGORY      39373 non-null  object
 5   PATIENT_ID_COUNT  39373 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 1.8+ MB


In [86]:
test_all

,PATIENT_SEX,MKB_CODE,ADRES,VISIT_MONTH_YEAR,AGE_CATEGORY,PATIENT_ID_COUNT
0,0,A00,Калининград,04.22,children,4
1,0,A00,Калининград,04.22,elderly,1
2,0,A00,Калининград,04.22,middleage,1
3,0,A00,Калининград,04.22,young,3
4,0,A01,Калининград,04.22,middleage,1
...,...,...,...,...,...,...
39368,1,Z96.6,Балтийск,04.22,elderly,1
39369,1,Z96.6,Гусев,04.22,middleage,1
39370,1,Z96.7,Гусев,04.22,young,1
39371,1,Z98.8,Озерск,04.22,children,1


In [87]:
# Если все ок, сохраняем сводную сабмиссию
test_all.to_csv('итоговый вариант.csv', sep=';', index=None)